<a href="https://colab.research.google.com/github/Vivek29112001/Machine-Learning-Projects/blob/main/CAR_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Understanding → Data Cleaning → Feature Engineering → Encoding → Scaling → Model Training → Evaluation

✅ STEP 1 — Understand Your Data
Ye tumhara dataset ka structure hai (columns):

['Company Names', 'Cars Names', 'Engines', 'CC/Battery Capacity',
 'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H',
 'Cars Prices', 'Fuel Types', 'Seats', 'Torque']
Tum model HorsePower predict karna chahte ho.

❗ Important Observation
Many columns text format me numbers + units mix hain.
Example:

Column	Sample Value	Issue
HorsePower	“963 hp”	Only number nikalna hoga
Engines	“V8”, “1.2L Petrol”	Categorical + mixed
CC/Battery Capacity	“3990 cc”, “1,200 cc”	Comma + unit remove
Total Speed	“340 km/h”	number nikalna
Performance	“2.5 sec”	number nikalna
Cars Prices	“$12,000-$15,000”	Price range + commas
Sabko numeric / clean format me convert karna hoga.

🔥 Step-by-Step Plan (Jaisa tumne bola)
Main har step ka code bhi bana dunga.

👉 STEP 2: Clean HorsePower Column
Goal → "963 hp" → 963
and "70-85 hp" → average (70+85)/2 = 77.5

👉 STEP 3: Clean CC/Battery Capacity
“3,982 cc” → 3982

👉 STEP 4: Convert Speed / Performance
“340 km/h” → 340
“2.5 sec” → 2.5

👉 STEP 5: Convert Price
Examples:

“$1,100,000” → 1100000

“$12,000-$15,000” → average (12000 + 15000)/2

👉 STEP 6: Encode Categorical Columns
Company Name → Label Encoding

Fuel Type → One Hot Encoding

Engines → Handle carefully (V6, V8, etc.)

👉 STEP 7: Train ML Model
Use Random Forest Regressor because:

Handles non-linear data

Works well with categorical+numeric mix

Good accuracy without tuning

👉 STEP 8: Evaluate
MAE

RMSE

R² Score

⚡ Next Step
Agar tum bolo → main pura cleaning code step-by-step chalate hue likh dunga, jisme:

✔ HorsePower cleaning
✔ CC cleaning
✔ Price cleaning
✔ Speed cleaning
✔ Feature engineering
✔ Train/test split
✔ Model training
✔ Prediction/saving



In [42]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [43]:
def extract_number_avg(s):
    if pd.isna(s):
        return np.nan
    s = str(s).lower()
    s = s.replace(",", "").strip()
    s = s.replace("cc", "").replace("km/h", "").replace("kmh", "").replace("sec", "").replace("seconds", "")
    s = s.replace("$", "").replace("rs", "").replace("₹", "").replace("nm", "")
    s = s.replace("hp", "").replace("bhp", "")
    if "-" in s:
        parts = [p.strip() for p in s.split("-") if p.strip()!='']
        nums = []
        for p in parts:
            m = re.findall(r"[-+]?\d*\.\d+|\d+", p)
            if m:
                nums.append(float(m[0]))
        if len(nums) >= 2:
            return float(sum(nums)/len(nums))
        elif len(nums)==1:
            return float(nums[0])
    m = re.findall(r"[-+]?\d*\.\d+|\d+", s)
    if m:
        return float(m[0])
    return np.nan

def clean_horsepower(val):
    return extract_number_avg(val)

def clean_cc(val):
    return extract_number_avg(val)

def clean_speed(val):
    return extract_number_avg(val)

def clean_performance(val):
    return extract_number_avg(val)

def clean_price(val):
    if pd.isna(val):
        return np.nan
    s = str(val).lower().replace(",", "").replace("usd", "").replace("inr", "").strip()
    return extract_number_avg(s)

def clean_torque(val):
    return extract_number_avg(val)

def clean_seats(val):
    return extract_number_avg(val)

In [44]:
train_path = "car_train.csv"
test_path = "car_test.csv"

train_df = pd.read_csv(train_path, encoding="latin1", low_memory=False)
test_df  = pd.read_csv(test_path, encoding="latin1", low_memory=False)

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain columns:", train_df.columns.to_list())
print("\nMissing values (train):\n", train_df.isnull().sum())
print("\nMissing values (test):\n", test_df.isnull().sum())

display(train_df.head())
display(test_df.head())


Train shape: (1218, 11)
Test shape: (244, 11)

Train columns: ['Company Names', 'Cars Names', 'Engines', 'CC/Battery Capacity', 'HorsePower', 'Total Speed', 'Performance(0 - 100 )KM/H', 'Cars Prices', 'Fuel Types', 'Seats', 'Torque']

Missing values (train):
 Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

Missing values (test):
 Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          0
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    1
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       0
dtype: int64


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,Porsche,Cayenne Turbo,4.0L V8 Twin Turbo,3996 cc,541 HP,286 km/h,4.0 sec,"$130,000",Petrol,5,770 Nm
1,HYUNDAI,Sonata,2.5L Inline-4,"2,500 cc",191 hp,240 km/h,8 sec,"$25,000",Petrol/Hybrid,5,250 Nm
2,LAMBORGHINI,AVENTADOR SVJ,V12,"6,498 cc",759 hp,350 km/h,2.8 sec,"$518,000",Petrol,2,720 Nm
3,Peugeot,5008 GT,Inline-4,"1,598 cc",180 hp,210 km/h,8.2 sec,"$40,000",Petrol,7,250 Nm
4,Porsche,Macan T,2.0L Turbo Inline-4,1984 cc,265 hp,233 km/h,6.2,"$63,000",Petrol,5,400 Nm


In [45]:
# --- Clean columns ---
df = train_df.copy()
df['HorsePower_Clean'] = df['HorsePower'].apply(clean_horsepower)
df['CC_Clean'] = df['CC/Battery Capacity'].apply(clean_cc)
df['TopSpeed_Clean'] = df['Total Speed'].apply(clean_speed)
df['Perf0_100_Clean'] = df['Performance(0 - 100 )KM/H'].apply(clean_performance)
df['Price_Clean'] = df['Cars Prices'].apply(clean_price)
df['Torque_Clean'] = df['Torque'].apply(clean_torque)
df['Seats_Clean'] = df['Seats'].apply(clean_seats)

In [46]:
df_model = df.dropna(subset=['HorsePower_Clean']).copy()
df_model.reset_index(drop=True, inplace=True)

In [47]:
feature_cols = ['Company Names', 'Engines', 'Fuel Types', 'CC_Clean', 'TopSpeed_Clean',
                'Perf0_100_Clean', 'Price_Clean', 'Torque_Clean', 'Seats_Clean']

X = df_model[feature_cols].copy()
y=df_model['HorsePower_Clean'].astype(float).copy()

print("X shape:", X.shape)
print("y shape:", y.shape)



X shape: (1218, 9)
y shape: (1218,)


In [51]:
def top_n_with_other(series, n=10):
    top = series.value_counts().nlargest(n).index.tolist()
    return series.apply(lambda x: x if x in top else 'Other')

X['Company_top'] = top_n_with_other(X['Company Names'], n=12)
X['Engine_top'] = top_n_with_other(X['Engines'], n=12)
X['Fuel_top'] = top_n_with_other(X['Fuel Types'], n=8)

X_num = X[['CC_Clean','TopSpeed_Clean','Perf0_100_Clean','Price_Clean','Torque_Clean','Seats_Clean']].copy()
X_num = X_num.fillna(X_num.median())

X_cat = pd.get_dummies(X[['Company_top','Engine_top','Fuel_top']], drop_first=True, dtype=float)

X_final = pd.concat([X_num.reset_index(drop=True), X_cat.reset_index(drop=True)], axis=1)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Pipeline + model
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1))
])

pipeline.fit(X_train, y_train)

# Evaluate
preds = pipeline.predict(X_val)
print("MAE:", mean_absolute_error(y_val, preds))
rmse = mean_squared_error(y_val, preds) ** 0.5
print("RMSE:", rmse)
print("R2:", r2_score(y_val, preds))

MAE: 20.92140368852459
RMSE: 65.50747271167388
R2: 0.8757142793646373


## Prepare test set and predict


In [54]:
t = test_df.copy()
t['HorsePower_Clean'] = t['HorsePower'].apply(clean_horsepower)
t['CC_Clean'] = t['CC/Battery Capacity'].apply(clean_cc)
t['TopSpeed_Clean'] = t['Total Speed'].apply(clean_speed)
t['Perf0_100_Clean'] = t['Performance(0 - 100 )KM/H'].apply(clean_performance)
t['Price_Clean'] = t['Cars Prices'].apply(clean_price)
t['Torque_Clean'] = t['Torque'].apply(clean_torque)
t['Seats_Clean'] = t['Seats'].apply(clean_seats)

t['Company_top'] = t['Company Names'].apply(lambda x: x if x in X['Company Names'].value_counts().nlargest(12).index else 'Other')
t['Engine_top'] = t['Engines'].apply(lambda x: x if x in X['Engines'].value_counts().nlargest(12).index else 'Other')
t['Fuel_top'] = t['Fuel Types'].apply(lambda x: x if x in X['Fuel Types'].value_counts().nlargest(8).index else 'Other')

t_num = t[['CC_Clean','TopSpeed_Clean','Perf0_100_Clean','Price_Clean','Torque_Clean','Seats_Clean']].copy()
t_num = t_num.fillna(X_num.median())

t_cat = pd.get_dummies(t[['Company_top','Engine_top','Fuel_top']], drop_first=True, dtype=float)

In [56]:
for c in X_cat.columns:
  if c not in t_cat.columns:
    t_cat[c] = 0.0
for c in list(t_cat.columns):
  if c not in X_cat.columns:
    t_cat.drop(c, axis=1, inplace=True)

t_final = pd.concat([t_num.reset_index(drop=True), t_cat.reset_index(drop=True)], axis=1)
t_final = t_final[X_final.columns]

test_preds = pipeline.predict(t_final)
t['Pred_HorsePower'] = test_preds
t.to_csv("submission.csv", index=False)
print("Saved predictions in submission.csv")

Saved predictions in submission.csv
